# Extrinsic Calibration

This notebook is an implementation, debugging and analysis of an Extrinsic Calibration Process using *CoppeliaSim* as a renderer. 

The goal is to implement and analyse, how all of the parameters and models used will affect the overall calibration and for how much. For this:
1. Make *CoppeliaSim's* clients that detects and sends blob coordinates detected in Vision Sensors' images;
2. Receive the messages asynchronously;
3. For each client, interpolate in time the detected blob positions;
4. Detect the calibration wand in the interpolated data;
5. Estimate the Fundamental and Essential Matrix, and it's scale;
6. Compute the relative pose between cameras.
---

In [1]:
# Importing modules...
import numpy as np
import scipy as sp
from scipy.optimize import linear_sum_assignment
import cv2
import socket

import sys
sys.path.append('../..') # Go back to base directory

from modules.plot.viewer3d import Viewer3D

from modules.vision.camera import Camera
from modules.vision.synchronizer import Synchronizer

from modules.integration.client import Client
from modules.integration.coppeliasim.server import CoppeliaSim_Server

# Instanciating `Server` and `Client` Structures

To wrap the information of all clients and mediate the communication between this notebook and *CoppeliaSim*, a `Server` object will be instanciated. 

The `Client` objects will be generated by their camera model and a synchronizer, representing their respective twin in the simulation.

For the `Server` instanciation, the following parameters must be given:
- Server address;
- *CoppeliaSim's* simulation Controller address;
- List containing all the clients present in the scene.

---

In [2]:
n_clients = 4 # Number of clients in the arena
clients = []  # Clients list

# Object matrix of Camera 0
base_matrix = np.array([[-7.07106781e-01,  5.00000000e-01, -5.00000000e-01, 2.50000000e+00],
                        [ 7.07106781e-01,  5.00000000e-01, -5.00000000e-01, 2.50000000e+00],
                        [ 1.46327395e-13, -7.07106781e-01, -7.07106781e-01, 2.50000000e+00]])

# Create clients
for ID in range(n_clients):
    # Spread all cameras uniformely in a circle around the arena
    R = np.array(sp.spatial.transform.Rotation.from_euler('z', (360 / n_clients) * ID, degrees=True).as_matrix())
    pose = np.vstack((R @ base_matrix,
                      np.array([0, 0, 0, 1])))

    # Generate associated camera model
    camera = (Camera(# Intrinsic Parameters
                     resolution=(1080,1080), 
                     fov_degrees=60.0,     
     
                     # Camera Pose
                     pose=pose,
     
                     # Rational Lens Distortion Model
                     # distortion_model='rational',
                     # distortion_coefficients=np.array([0.014, -0.003, -0.0002, -0.000003, 0.0009, 0.05, -0.007, 0.0017]),
 
                     # Fisheye Lens Distortion Model
                     distortion_model='fisheye',
                     distortion_coefficients=np.array([0.395, 0.633, -2.417, 2.110]),
     
                     # Image Noise Model
                     snr_dB=13
                     ))
    
    clients.append(Client(camera=camera))

In [3]:
# Create server
server = CoppeliaSim_Server(clients=clients,
                            server_address=('127.0.0.1', 8888),
                            controller_address=('127.0.0.1', 7777))

# Requesting Scene

A **Scene Request** will send to *CoppeliaSim* the data necessary to instanciate the simulated clients' twins in the childscripts. The scene request can be used to reset the client's data from the server if called again.

---

In [4]:
# Request scene with the associated server clients
if not server.request_scene():
    sys.exit() # Scene request failed!

[SERVER] Wrapping up CoppeliaSim scene info
[SERVER] Scene info sent
[SERVER] Scene set!


# Requesting Capture

A **Capture Request** will trigger a simulation in *CoppeliaSim* sending the total simulation time of the requested capture. Once the simulation stops, another capture request can be called for another simulation

In the simulation start, the clients will be created and send their ID to the server for client registration.

---

In [13]:
# Capture specifications
blob_count = 3 # Number of expected markers
capture_time = 30.0 # In seconds
window = 3 # The minimum ammount of points for interpolating 
throughput = 20 # Triangulated scenes per second
step = 1 / throughput # Interpolation timestep

# Capture synchronizer
synchronizer = Synchronizer(blob_count, window, step, capture_time)

# Request capture (start simulation)
if not server.request_capture(synchronizer):
    sys.exit() # Capture request failed!

# Wait for client identification
server.register_clients()

[SERVER] Capture info sent
[SERVER] Capture confirmed!
[SERVER] Waiting for clients...
	Client 0 registered
	Client 1 registered
	Client 2 registered
	Client 3 registered
[SERVER] All clients registered!


# Running Simulation

The messages will be received here in the following loop until a server timeout is reached. To analyse the content of each message, toggle the `verbose` flag. 

The loop will wait for a message to be received by the socket. When a message comes, it will be recorded in their client's `message_log` and wait for the next message. 

Since the calibration prioritizes the amount of quality data and not real time triangulation, the in-loop actions will be post-processed to avoid message losses.

---

In [14]:
verbose = False

timeout = 5 # In seconds
server.udp_socket.settimeout(timeout) # Set server timeout
print(f'[SERVER] Timeout set to {timeout} seconds\n')

# Breaks in the timeout
while True: 
    # Wait for message - Event guided!
    try:
        message_bytes, address = server.udp_socket.recvfrom(server.buffer_size)

    except socket.timeout as err:
        print('\n[SERVER] Timed Out!')
        
        break # Close capture loop due to timeout
    
    # Check if client exists
    try:
        ID = server.client_addresses[address] # Client Identifier
    
    except:
        if verbose: print('> Client not recognized')

        continue # Jump to wait for the next message
    
    # Show sender
    if verbose: print(f'> Received message from Client {ID} ({address[0]}, {address[1]})')

    # Save message
    server.clients[ID].message_log.append(message_bytes)

[SERVER] Timeout set to 5 seconds


[SERVER] Timed Out!


# Post-processing Data

For each client, the code will loop through it's message history and it will:
1. Decode message; 
2. Parse the message for it's contents;
3. Check if the message is valid;
    - A valid message is composed of a blob coodinate and it's area (per blob) and the PTS of the message.
4. Undistort blob data
5. Save data in the `Synchronizer` structure.

---

In [15]:
verbose = False

for client in server.clients:
    # Parse through client's message history
    for message_bytes in client.message_log: 

        # Decode message
        try:
            message = np.frombuffer(message_bytes, dtype=np.float32)

        except:
            if verbose: print('> Couldn\'t decode message')

            continue # Jump to the next message

        # Empty message
        if not message.size:
            if verbose: print('\tEmpty message')

            continue # Jump to the next message

        # Extracting the message's PTS
        PTS = message[-1] # Last element of the message 

        # Valid message is [u, v, A] per blob and the PTS of the message
        if message.size !=  3 * blob_count + 1:

            if message.size == 1: # Only PTS
                if verbose: print(f'\tNo blobs were detected - {PTS :.3f} s')

            else: 
                if verbose: 
                    print(f'\tWrong blob count or corrupted message')
                    print(f'Corrupted Message: {message}')

            continue # Jump to the next message

        # Extracting blob data (coordinates & area)
        sync_blobs_pair = message[:-1].reshape(-1, 3) # All but last element (reserved for PTS)

        # Extracting centroids
        blob_centroids = sync_blobs_pair[:,:2] # Ignoring their area

        # Undistorting blobs centroids
        undistorted_blobs = client.camera.undistort_points(blob_centroids)          

        # Print blobs
        if verbose:
            print(f'\tDetected Blobs - {PTS :.3f} s')
            print('\t' + str(sync_blobs_pair).replace('\n', '\n\t'))

        # Save data
        valid_data = client.synchronizer.add_data(undistorted_blobs, PTS)

        if verbose: 
            if valid_data:
                print('\tData Accepted!')
            else:
                print('\tData Refused!')

# Matching Blobs to Markers

For the Fundamental Matrix estimation algorithm to work, points must be matched from an image to another. 

Since the Fundamental Matrix is still not known, it means the points cannot be ordered by epipolar lines. 

To work around that, the markers in the scene will be arranged in a collinear pattern, where the distance between adjacent markers in the line will be in an arbitrary non-unit ratio.

The reason for they to be collinear is the fact that in an undistorted image the distance ratio between the detected blobs will still be the same, but in pixels.

The ratio will make it possible to label each blob to each marker. The below algorithm will order the blobs according to the ratio in the following order:
1. The closest blob to the middle blob;
2. The middle blob;
3. The furthest blob from the middle blob;

---

In [16]:
def collinear_order(blobs, wand_ratio):
    # Distances between blobs
    distances = np.array([np.linalg.norm(blobs[0] - blobs[1]), 
                          np.linalg.norm(blobs[1] - blobs[2]), 
                          np.linalg.norm(blobs[2] - blobs[0])])
    
    # Shortest distance
    shortest = np.min(distances)

    # If x is invalid
    if shortest == 0.0 or shortest is np.nan:
        return None

    # Normalize distances
    distances /= shortest

    # Measured unique distance sums
    measured_unique_sums = np.array([distances[0] + distances[2],
                                     distances[0] + distances[1],
                                     distances[1] + distances[2]])
    
    # Exprected unique distance sums
    expected_unique_sums = np.array([wand_ratio[0] + wand_ratio[0] + wand_ratio[1],
                                     wand_ratio[0] + wand_ratio[1],
                                     wand_ratio[0] + wand_ratio[1] + wand_ratio[1]])
    
    # Error matrix
    difference_matrix = np.array([[np.abs(measured - expected)
                                   for measured in measured_unique_sums] 
                                   for expected in expected_unique_sums])

    # Using the hungarian (Munkres) assignment algorithm to find unique correspondences between blobs and epilines
    _, new_indices = linear_sum_assignment(difference_matrix)

    return blobs[new_indices]

# Measured distances between markers
# Distances: [AB, BC, AC]
wand_distances = np.array([5e-2, 10e-2, 15e-2]) # In meters
wand_ratio = (1.0, wand_distances[1] / wand_distances[0])

# Playing Back a Camera's Feed

The following cell will replicate a real time camera feed of the simulation. Change the `ID` parameter to switch between camera views.

In this case, the blob labelling will be shown in the feed to check if the ordering algorithm is working as expected.

---

In [17]:
# Playback a camera image feed in fidelity time
playback = False

if playback:
    ID = 0 # Camera ID to be watched
        
    # Converting to np arrays
    sync_PTS = np.array(server.clients[ID].synchronizer.sync_PTS)
    sync_blobs_pair = np.array(server.clients[ID].synchronizer.sync_blobs)
    
    # Generating frames
    images = []
    for PTS, blobs in zip(sync_PTS, sync_blobs_pair):
        image = np.zeros(server.clients[ID].camera.resolution)
        image = cv2.putText(image, str(PTS), 
                                    [40, 40], 
                                    cv2.FONT_HERSHEY_SIMPLEX, 
                                    1, 
                                    (255, 0, 0), 
                                    1, 
                                    cv2.LINE_AA)

        ordered_blobs = collinear_order(blobs, (1, 2))

        if ordered_blobs is not None:
            for tag, blob in zip(['A', 'B', 'C'], ordered_blobs):

                image = cv2.putText(image, tag, 
                                    blob.astype(int), 
                                    cv2.FONT_HERSHEY_SIMPLEX, 
                                    0.5, 
                                    (255, 255, 255), 
                                    1, 
                                    cv2.LINE_AA)
            
        images.append(image)

    # Getting delay between each frame
    delays = sync_PTS[1:] - sync_PTS[:-1]

    # Playing the animation
    for delay, image in zip(delays, images):
        cv2.imshow(f'Camera {ID} Feed', image)
        cv2.waitKey(int(1e3 * delay))

    # Closing all open windows 
    cv2.destroyAllWindows()

# Estimating Relative Poses

For each pair, the following process will be done to estimate their relative rotation and translation:
1. Order captured blobs in the proposed collinear order;
2. Estimate the Fundamental Matrix using OpenCV's function;
3. Calculate the Essential Matrix using using the estimated Fundamental Matrix;
4. Decompose the Essential Matrix into a Rotation Matrix and Unit Translation Vector;
5. Calculate the Projection Matrix of each matrix and triangulate the wand markers in each frame;
6. For each frame, relate the calculated unscaled marker distances to the measured marker distances and get the respective scale of the camera pair;
7. Build the estimated extrinsic matrix for each pair;

---

In [18]:
def decompose_essential_matrix(E, 
                               blobs_reference, 
                               blobs_auxiliary, 
                               intrinsic_matrix_reference, 
                               intrinsic_matrix_auxiliary):
    
    # Decompose the essential matrix
    R1, R2, t = cv2.decomposeEssentialMat(E)
    
    # Possible nase changes from the reference frame to
    R_t_options = [[R1,  t],
                   [R1, -t],
                   [R2,  t],
                   [R2, -t]]
    
    best_option = None

    # Projection matrix of the first camera
    P_reference = intrinsic_matrix_reference @ np.eye(4)[:3, :4]
    
    # Triangulate points and check their validity
    for option, R_t in enumerate(R_t_options):
        P_auxiliary = intrinsic_matrix_auxiliary @ np.hstack(R_t)

        triangulated_points_4D = cv2.triangulatePoints(P_reference.astype(np.float32), 
                                                       P_auxiliary.astype(np.float32), 
                                                       blobs_reference.T.astype(np.float32), 
                                                       blobs_auxiliary.T.astype(np.float32))
        
        # Normalize homogeneous coordinates and discard last row
        triangulated_points_3D = (triangulated_points_4D / triangulated_points_4D[-1])[:-1, :]

        # Count how many points have a positive z-coordinate 
        # This indicates if the point is in front of the camera or not
        frontal_points = np.count_nonzero(triangulated_points_3D[2] > 0)

        # No markers were shot behind the cameras
        if frontal_points == triangulated_points_3D.shape[1]:
            best_option = option # Where all points are frontal points

    # No valid decomposition
    if best_option is None:
        return None, None 
    
    # Valid decomposition
    return R_t_options[best_option]

In [19]:
# All possible camera pairs
client_ids = np.arange(n_clients)
pairs = [(i, j) for i in client_ids for j in client_ids[client_ids != i]] 

relative_poses = {}
triangulated_markers = {}

for pair in pairs:
    # Getting data from pair
    reference, auxiliary = pair
    
    # Synchronized blobs for each pair
    sync_blobs_pair = [server.clients[ID].synchronizer.sync_blobs for ID in pair]
    
    # Order collinear blobs
    ordered_blobs_reference = []
    ordered_blobs_auxiliary = []

    for sync_blobs_reference, sync_blobs_auxiliary in zip(*sync_blobs_pair):
        blobs_reference = collinear_order(sync_blobs_reference, wand_ratio)
        blobs_auxiliary = collinear_order(sync_blobs_auxiliary, wand_ratio)

        if blobs_reference is None or blobs_auxiliary is None:
            continue

        ordered_blobs_reference.append(blobs_reference)
        ordered_blobs_auxiliary.append(blobs_auxiliary)

    ordered_blobs_reference = np.array(ordered_blobs_reference)
    ordered_blobs_auxiliary = np.array(ordered_blobs_auxiliary)

    # Join all ordered blobs in a single matrix
    all_blobs_reference = np.vstack(ordered_blobs_reference)
    all_blobs_auxiliary = np.vstack(ordered_blobs_auxiliary)
    
    # Ground truth fundamental matrix
    F_ground_truth = server.multiple_view.fundamental_matrix[reference][auxiliary]

    F_estimated, mask = cv2.findFundamentalMat(points1=all_blobs_reference, 
                                               points2=all_blobs_auxiliary, 
                                               method=cv2.FM_8POINT)

    print(f'Pair {pair}:')
    print(f'> Matrix Distance of the Estimated F to the Ground Truth F = {np.abs(np.linalg.det(F_ground_truth - F_estimated)):.5e}')

    # Selecting inlier points    
    inlier_all_blobs_reference = all_blobs_reference[mask.ravel() == 1]
    inlier_all_blobs_auxiliary = all_blobs_auxiliary[mask.ravel() == 1]

    mask = np.array([np.prod(flags) for flags in mask.reshape(-1, 3)])

    inlier_blobs_reference = ordered_blobs_reference[mask == 1]
    inlier_blobs_auxiliary = ordered_blobs_auxiliary[mask == 1]

    # Calculating essential matrix
    E = server.clients[auxiliary].camera.intrinsic_matrix.T @ F_estimated @ server.clients[reference].camera.intrinsic_matrix

    # Decomposing essential matrix
    R, t = decompose_essential_matrix(E,
                                      inlier_all_blobs_reference,
                                      inlier_all_blobs_auxiliary,
                                      server.clients[reference].camera.intrinsic_matrix,
                                      server.clients[auxiliary].camera.intrinsic_matrix)

    # Check if decomposition worked
    if R is None and t is None:
        print('> No valid decomposition!')
        continue

    # Calculating projection matrices
    # The reference camera will be the reference frame, thus the identity matrix
    P_reference = server.clients[reference].camera.intrinsic_matrix @ np.eye(4)[:3, :4]
    P_auxiliary = server.clients[auxiliary].camera.intrinsic_matrix @ np.hstack((R, t))
    
    all_triangulated_points_3D = [] # List of triangulated points
    all_unscaled_distances = [] # List of all measured distance
    scales = [] # Scale factor for each triangulated frame

    for points_reference_PTS, points_auxiliary_PTS in zip(inlier_blobs_reference, inlier_blobs_auxiliary):
        triangulated_points_4D = cv2.triangulatePoints(P_reference.astype(np.float32), 
                                                       P_auxiliary.astype(np.float32), 
                                                       points_reference_PTS.T.astype(np.float32), 
                                                       points_auxiliary_PTS.T.astype(np.float32))
        
        # Normalize homogeneous coordinates and discard last row
        triangulated_points_3D = (triangulated_points_4D / triangulated_points_4D[-1])[:-1, :]

        # Unscaled distances
        unscaled_distances = np.array([np.linalg.norm(triangulated_points_3D.T[0] - triangulated_points_3D.T[1]),
                                       np.linalg.norm(triangulated_points_3D.T[1] - triangulated_points_3D.T[2]),
                                       np.linalg.norm(triangulated_points_3D.T[0] - triangulated_points_3D.T[2])])

        # Save values
        all_triangulated_points_3D.append(triangulated_points_3D)
        all_unscaled_distances.append(unscaled_distances)
        scales.append(np.sum(wand_distances) / np.sum(unscaled_distances))

    # Mean scale factor
    scale = np.mean(np.array(scales))
    print(f'> Mean K = {scale:.5f}')

    # Scaling data
    unscaled_distances_mean = np.mean(np.array(all_unscaled_distances), axis=0)
    scaled_distances_mean = unscaled_distances_mean * scale

    unscaled_distances_std = np.std(np.array(all_unscaled_distances), axis=0)
    scaled_distances_std = unscaled_distances_std * scale

    scaled_triangulated_points_3D = np.hstack(np.array(all_triangulated_points_3D)) * scale
    triangulated_markers[pair] = scaled_triangulated_points_3D

    print(f'> Measured AB = {scaled_distances_mean[0]:.5f} m | Std deviation = {scaled_distances_std[0]:.5f} m')
    print(f'> Measured BC = {scaled_distances_mean[1]:.5f} m | Std deviation = {scaled_distances_std[1]:.5f} m')
    print(f'> Measured AC = {scaled_distances_mean[2]:.5f} m | Std deviation = {scaled_distances_std[2]:.5f} m')
    print()

    # Saving scaled matrix
    extrinsic_matrix_auxiliary = np.vstack((np.hstack((R, t * scale)),
                                            np.array([0, 0, 0, 1])))
    relative_poses[pair] = np.linalg.inv(extrinsic_matrix_auxiliary)

Pair (0, 1):
> Matrix Distance of the Estimated F to the Ground Truth F = 2.41446e-16
> Mean K = 5.00666
> Measured AB = 0.05003 m | Std deviation = 0.00288 m
> Measured BC = 0.10007 m | Std deviation = 0.00164 m
> Measured AC = 0.14999 m | Std deviation = 0.00213 m

Pair (0, 2):
> Matrix Distance of the Estimated F to the Ground Truth F = 6.09875e-16
> Mean K = 7.07593
> Measured AB = 0.04996 m | Std deviation = 0.00160 m
> Measured BC = 0.10017 m | Std deviation = 0.00273 m
> Measured AC = 0.15009 m | Std deviation = 0.00366 m

Pair (0, 3):
> Matrix Distance of the Estimated F to the Ground Truth F = 4.58614e-16
> Mean K = 5.04061
> Measured AB = 0.04986 m | Std deviation = 0.00168 m
> Measured BC = 0.10023 m | Std deviation = 0.00243 m
> Measured AC = 0.15005 m | Std deviation = 0.00322 m

Pair (1, 0):
> Matrix Distance of the Estimated F to the Ground Truth F = 2.41446e-16
> Mean K = 5.00667
> Measured AB = 0.05003 m | Std deviation = 0.00288 m
> Measured BC = 0.10007 m | Std devia

# Displaying Calibrated Camera Poses

To visually inspect if the results are coherent to the scene, the calibrated camera poses will be displayed in a 3D viewer. The 0th camera will be shown as the reference frame.

Choose the Pair ID to show the respective triangulated points of each pair.

---

In [20]:
# Create the Scene Viewer
scene = Viewer3D(title='Calibrated Camera Poses', 
                 size=10)

reference = 0 # Reference frame camera

# Add camera frames to the scene
scene.add_frame(np.eye(4), f'Camera {reference}', axis_size=0.4)
for ID in client_ids[client_ids != reference]: 
    scene.add_frame(relative_poses[(reference, ID)], f'Camera {ID}', axis_size=0.4)

pair = (0, 1) # Triangulation pair

# Add triangulated markers to the scene
scene.add_points(triangulated_markers[pair], f'Triangulation by Pair {pair}')

# Plot scene
scene.figure.show(renderer='notebook_connected')